In [2]:
import pandas as pd
import os
from pathlib import Path

In [3]:
data_folder_path = Path('../../../data/')
# print(os.listdir(data_folder_path))

In [4]:
data_continuous = pd.read_csv(data_folder_path / 'data_train_continuous_data.csv')
print(len(data_continuous))

255606


In [5]:
data_n_grams = pd.read_csv(data_folder_path / 'data_train_n_grams.csv')
print(len(data_n_grams))

255606


In [6]:
# Define common columns that should be merged on
common_cols = ['spotify_song_id', 'chords', 'simplified_chords', 'decade', 'main_genre']

merged_data = data_continuous.merge(data_n_grams, on=common_cols, how='inner')

print(f"Merged data shape: {merged_data.shape}")
print(f"Columns: {len(merged_data.columns)}")

Merged data shape: (255606, 142)
Columns: 142


In [7]:
# Check if they have the same values
are_same = merged_data['spotify_song_id'].equals(merged_data['spotify_track_id'])
print(f"Are spotify_song_id and spotify_track_id the same? {are_same}")

# Check for any differences
if not are_same:
    differences = merged_data[merged_data['spotify_song_id'] != merged_data['spotify_track_id']]
    print(f"\nNumber of rows with differences: {len(differences)}")
    print(f"Total rows: {len(merged_data)}")

# Check if all values match
matching_count = (merged_data['spotify_song_id'] == merged_data['spotify_track_id']).sum()
print(f"\nMatching values: {matching_count}/{len(merged_data)}")

print(f"\nThe 'spotify_track_id' column contains nans for rows that do not have Spotify data")

Are spotify_song_id and spotify_track_id the same? False

Number of rows with differences: 73498
Total rows: 255606

Matching values: 182108/255606

The 'spotify_track_id' column contains nans for rows that do not have Spotify data


In [8]:
# Create a DataFrame with column names
columns_df = pd.DataFrame({'column_name': merged_data.columns})

# Define your column lists
id_cols = ['spotify_song_id', 'spotify_track_id']
data_cols = ['chords_x', 'simplified_chords_x', 'simplified_chords', 'spotify_song_id', 'chords_y', 'missing_notes', 'simplified_chords_y', 'chords', 'spotify_artist_id', 'spotify_success', 'spotify_duration_ms', 'spotify_artist_popularity', 'spotify_followers', 'spotify_track_name', 'spotify_artists', 'spotify_album_name', 'spotify_release_date', 'spotify_artist_name', 'spotify_genres']

target_cols = ['decade', 'main_genre', 'spotify_popularity']
predictor_continuous = ['drone_ratio', 'average_overlap', 'average_2overlap', 'average_3overlap', 'average_4overlap', 'average_5overlap', 'maj_triad_ratio', 'min_triad_ratio', 'unique_5gram_density', 'unique_chord_density']

# Create function to categorize columns
def categorize_column(col):
    if col in id_cols:
        return 'id'
    elif col in data_cols:
        return 'data'
    elif col in target_cols:
        return 'target'
    elif col in predictor_continuous:
        return 'predictor_continuous'
    elif col.startswith('contain'):
        return 'predictor_n_grams'
    else:
        return 'unknown'

# Create DataFrame with column names and types
columns_df = pd.DataFrame({
    'column_name': merged_data.columns,
    'column_type': [categorize_column(col) for col in merged_data.columns]
})

print(columns_df['column_type'].value_counts())
print(f"\nTotal columns: {len(columns_df)}")


column_type
predictor_n_grams       109
data                     18
predictor_continuous     10
target                    3
id                        2
Name: count, dtype: int64

Total columns: 142


In [9]:
# Add the hot 100 index to the data

data_hot_100 = pd.read_csv(data_folder_path / 'spotify_hot100.csv')
data_hot_100 = pd.read_csv(data_folder_path / 'spotify_hot100.csv', usecols=['track_id', 'on_hot100'])

print(data_hot_100.columns)

merged_data_final = merged_data.merge(
    data_hot_100,
    how='left',
    left_on='spotify_track_id',
    right_on='track_id'
)
print(len(merged_data_final))

# add to column csv
new_row = {'column_name': 'track_id', 'column_type': 'id'}
columns_df = pd.concat([columns_df, pd.DataFrame([new_row])], ignore_index=True)
new_row = {'column_name': 'on_hot100', 'column_type': 'target'}
columns_df = pd.concat([columns_df, pd.DataFrame([new_row])], ignore_index=True)

Index(['track_id', 'on_hot100'], dtype='object')
255606


In [10]:
# Define the desired order of column types
type_order = ['id', 'data', 'target', 'predictor_continuous', 'predictor_n_grams']

# Create a categorical type with the specified order
columns_df['column_type'] = pd.Categorical(columns_df['column_type'], categories=type_order, ordered=True)

# Sort by column_type
columns_df = columns_df.sort_values('column_type').reset_index(drop=True)

In [11]:
# Save to CSV
output_path = data_folder_path / 'data_train_all_columns.csv'
columns_df.to_csv(output_path, index=False)

print(f"Saved {len(merged_data.columns)} column names to {output_path}")

Saved 142 column names to ../../../data/data_train_all_columns.csv


In [12]:
# Save merged data to CSV
output_path = data_folder_path / 'data_train_all.csv'
merged_data_final.to_csv(output_path, index=False, na_rep='NaN')
print(f"Saved merged data to {output_path}")

Saved merged data to ../../../data/data_train_all.csv
